## Data adquisition

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
from scipy.stats import boxcox

warnings.filterwarnings('ignore')

# Download stock data
FinIns = yf.download('NG=F', start='2021-12-22', end='2025-01-30')
FinIns_close = FinIns[['Close']]

# Convert index to datetime and set business day frequency
FinIns_close.index = pd.to_datetime(FinIns_close.index)
FinIns_close = FinIns_close.asfreq('B')

# Forward fill missing values
FinIns_close = FinIns_close.fillna(method='ffill')
FinIns_close.dropna(inplace=True)

# Ensure all values are strictly positive for Box-Cox
FinIns_close.dropna(inplace=True)  # Ensure no NaN values
min_close = FinIns_close['Close'].min().item()  # Extract scalar value

if min_close <= 0:
    FinIns_close['Close'] += abs(min_close) + 1  # Shift all values to be positive


# Transformations
FinIns_close['Log_Close'] = np.log(FinIns_close['Close'])
FinIns_close['Square_Close'] = FinIns_close['Close'] ** 2
FinIns_close['BoxCox_Close'], lambda_bc = boxcox(FinIns_close['Close'].to_numpy().flatten())

# Display transformed data
print(FinIns_close.head())

# Plot transformed data
fig, axes = plt.subplots(4, 1, figsize=(12, 12))

axes[0].plot(FinIns_close['Close'], label='Data')
axes[0].set_title('Normal Data')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Closing Price')
axes[0].legend()

axes[1].plot(FinIns_close['Square_Close'], label='Square Transformation', color='orange')
axes[1].set_title('Square Transformation')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Squared Closing Price')
axes[1].legend()

axes[2].plot(FinIns_close['BoxCox_Close'], label='Box-Cox Transformation', color='green')
axes[2].set_title('Box-Cox Transformation')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Box-Cox Transformed Closing Price')
axes[2].legend()

axes[3].plot(FinIns_close['Log_Close'], label='Log Transformation')
axes[3].set_title('Logarithmic Transformation')
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Log(Closing Price)')
axes[3].legend()

plt.tight_layout()
plt.show()


## Time Series type

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Plot ACF and PACF for the original  stock closing prices
plt.figure(figsize=(10, 4))
plt.subplot(211)
plot_acf(FinIns['Close'], ax=plt.gca(), zero=False, lags=40)
plt.subplot(212)
plot_pacf(FinIns['Close'], ax=plt.gca(), zero=False, lags=40)
plt.tight_layout()
plt.show()
plt.show()

# Function to perform Augmented Dickey-Fuller (ADF) Test
def adf_test(series, title=''):
    """
    Perform Augmented Dickey-Fuller Test on a time series.

    Parameters:
    - series (pd.Series): The time series data to be tested.
    - title (str): Optional title for the output.

    Returns:
    - dict: ADF test results including test statistic, p-value, lags used, and critical values.
    """
    try:
        print(f'\n{"="*40}\nAugmented Dickey-Fuller Test: {title}\n{"="*40}')
        result = adfuller(series.dropna(), autolag='AIC')  # Handle NaN values
        output = {
            'ADF Test Statistic': result[0],
            'p-value': result[1],
            '# Lags Used': result[2],
            'Number of Observations Used': result[3],
            'Critical Values': result[4]
        }
        
        for key, value in result[4].items():
            print(f'Critical Value ({key}): {value:.4f}')

        print(f'ADF Test Statistic: {result[0]:.4f}')
        print(f'p-value: {result[1]:.4f}')
        print(f'# Lags Used: {result[2]}')
        print(f'Number of Observations Used: {result[3]}')

        if result[1] <= 0.05:
            print("✅ The series is stationary (reject H0).")
        else:
            print("❌ The series is non-stationary (fail to reject H0).")
        
        return output

    except Exception as e:
        print(f"Error in ADF test: {e}")
        return None

# Load financial data
FinIns = yf.download('NG=F', start='2021-12-22', end='2025-01-30')
FinIns_close = FinIns[['Close']].dropna()  # Remove NaN values

# Perform ADF test on original closing prices
adf_test(FinIns_close['Close'], title='Financial Instrument Closing Prices - Original')

# Compute differenced data for stationarity
FinIns_close['Close_diff'] = FinIns_close['Close'].diff().dropna()

# Perform ADF test on differenced data
adf_test(FinIns_close['Close_diff'], title='Financial Instrument Closing Prices - Differenced')

# Plot the differenced data
plt.figure(figsize=(12, 6))
plt.plot(FinIns_close['Close_diff'], label='Differenced Data', color='purple')
plt.title('Differenced Closing Prices')
plt.xlabel('Date')
plt.ylabel('Differenced Closing Prices')
plt.legend()
plt.grid()
plt.show()

# Perform seasonal decomposition
try:
    result_decomp = seasonal_decompose(FinIns_close['Close'].dropna(), model='additive', period=30)  # Assuming monthly seasonality
    result_decomp.plot()
    plt.show()
except Exception as e:
    print(f"Error in seasonal decomposition: {e}")



## Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
from scipy.stats import boxcox
import matplotlib.dates as mdates  # ✅ Import for better date formatting

warnings.filterwarnings('ignore')

# Load financial data
FinIns = yf.download('NG=F', start='2021-12-22', end='2025-01-30')
FinIns_close = FinIns[['Close']].dropna().sort_index()

# Ensure business day frequency
FinIns_close = FinIns_close.asfreq('B').interpolate()

# Split the data into train and test
train = FinIns_close.iloc[:-24].copy()
test = FinIns_close.iloc[-24:].copy()

# 1️⃣ Naive Forecast
def naive_forecast(train, test):
    forecast = test.copy()
    forecast['naive'] = train.iloc[-1]['Close']
    return forecast

y_hat_naive = naive_forecast(train, test)

# 2️⃣ Seasonal Naive Forecast
def seasonal_naive_forecast(train, test, season_length=12):
    forecast = test.copy()
    train_values = train['Close'].values
    forecast['snaive'] = [train_values[-season_length + i % season_length] for i in range(len(test))]
    return forecast

y_hat_snaive = seasonal_naive_forecast(train, test)

# 3️⃣ Mean Forecast
def mean_forecast(train, test):
    forecast = test.copy()
    forecast['mean'] = train['Close'].mean()
    return forecast

y_hat_mean = mean_forecast(train, test)

# 4️⃣ Drift Forecast (FINAL FIX)
def drift_forecast(train, test):
    forecast = test.copy()
    num_periods = len(train) - 1  # Avoid division by zero
    
    if num_periods > 0:
        last_close = train['Close'].iloc[-1].item()  # Ensure scalar float
        first_close = train['Close'].iloc[0].item()  # Ensure scalar float
        
        drift = (last_close - first_close) / num_periods
        drift_values = last_close + np.arange(1, len(test) + 1) * drift
    else:
        last_close = train['Close'].iloc[-1].item()  # Ensure scalar float
        drift_values = np.full(len(test), last_close)  # Constant forecast
    
    # ✅ Convert NumPy array to Pandas Series with correct index
    forecast['drift'] = pd.Series(drift_values, index=test.index)
    
    return forecast

# Run the fixed function
y_hat_drift = drift_forecast(train, test)

# Ensure index types are correct
for df in [train, test, y_hat_naive, y_hat_snaive, y_hat_mean, y_hat_drift]:
    df.index = pd.to_datetime(df.index)

# Extract last year's data dynamically
last_year_start = max(train.index[-1] - pd.DateOffset(years=1), train.index[0])
train_last_year = train.loc[last_year_start:]
test_last_year = test

y_hat_naive_last = y_hat_naive.loc[last_year_start:]
y_hat_snaive_last = y_hat_snaive.loc[last_year_start:]
y_hat_mean_last = y_hat_mean.loc[last_year_start:]
y_hat_drift_last = y_hat_drift.loc[last_year_start:]

# ✅ Figure 1: Full Dataset Forecasts
plt.figure(figsize=(14, 6))  # ✅ Create a new figure

# ✅ Plot actual data
plt.plot(train.index, train['Close'], label='Train', color='blue')
plt.plot(test.index, test['Close'], label='Test', color='grey', linewidth=2, alpha=0.7)

# ✅ Plot forecasts
plt.plot(y_hat_naive.index, y_hat_naive['naive'], label='Naive Forecast', color='red', linestyle='--', alpha=0.8)
plt.plot(y_hat_snaive.index, y_hat_snaive['snaive'], label='Seasonal Naive Forecast', color='green', linestyle='--', alpha=0.8)
plt.plot(y_hat_mean.index, y_hat_mean['mean'], label='Mean Forecast', color='orange', linestyle='--', alpha=0.8)
plt.plot(y_hat_drift.index, y_hat_drift['drift'], label='Drift Forecast', color='purple', linestyle='--', alpha=0.8)

# ✅ Ensure x-axis displays the full dataset range
full_start = min(train.index.min(), test.index.min(), y_hat_naive.index.min(), y_hat_snaive.index.min(),
                 y_hat_mean.index.min(), y_hat_drift.index.min())  # ✅ Get the earliest date
full_end = max(train.index.max(), test.index.max(), y_hat_naive.index.max(), y_hat_snaive.index.max(),
               y_hat_mean.index.max(), y_hat_drift.index.max())  # ✅ Get the latest date
plt.xlim([full_start, full_end])  # ✅ Set full range manually

# ✅ Set up correct date formatting
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))  # ✅ Show one major tick per year
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator(interval=3))  # ✅ Show minor ticks every 3 months
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # ✅ Format as "2022", "2023", etc.
plt.xticks(rotation=45)  # ✅ Rotate labels for better readability

# ✅ Add title and labels
plt.title('Forecasting Benchmarks: Full Dataset', fontsize=16)
plt.ylabel('Closing Price', fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)

# ✅ Show the first plot
plt.show()

# ✅ Figure 2: Last Year Forecasts (Zoomed)
plt.figure(figsize=(14, 6))  # ✅ Create a separate figure for zoomed-in data

# ✅ Extract last year's range
last_year_start = max(test.index[-1] - pd.DateOffset(years=1), train.index[0])  # Ensure valid range
last_train = train.loc[train.index >= last_year_start] if not train.empty else pd.DataFrame()
last_test = test.loc[test.index >= last_year_start] if not test.empty else pd.DataFrame()

print(f"Test Data Available: {not last_test.empty}")  # ✅ Debugging statement

# ✅ Plot actual data (Train & Test Split)
if not last_train.empty:
    plt.plot(last_train.index, last_train['Close'], label='Train', color='blue')
if not last_test.empty:
    plt.plot(last_test.index, last_test['Close'], label='Test', color='grey', linewidth=2, alpha=0.7)  # ✅ Fix applied

# ✅ Plot forecasts
plt.plot(y_hat_naive.loc[last_year_start:].index, y_hat_naive.loc[last_year_start:]['naive'], 
         label='Naive Forecast', color='red', linestyle='--')
plt.plot(y_hat_snaive.loc[last_year_start:].index, y_hat_snaive.loc[last_year_start:]['snaive'], 
         label='Seasonal Naive Forecast', color='green', linestyle='--')
plt.plot(y_hat_mean.loc[last_year_start:].index, y_hat_mean.loc[last_year_start:]['mean'], 
         label='Mean Forecast', color='orange', linestyle='--')
plt.plot(y_hat_drift.loc[last_year_start:].index, y_hat_drift.loc[last_year_start:]['drift'], 
         label='Drift Forecast', color='purple', linestyle='--')

# ✅ Ensure x-axis displays only last year's range
plt.xlim([last_year_start, test.index[-1]])

# ✅ Set up correct date formatting
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # ✅ Show one tick per month
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  # ✅ Format as "Jan 2024", "Feb 2024", etc.
plt.xticks(rotation=45)  # ✅ Rotate labels for better readability

# ✅ Add title and labels
plt.title('Benchmark Forecasts (Last Year) with Train/Test Split', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Closing Price', fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)

# ✅ Show the second plot
plt.show()


# ✅ Show the second plot
plt.show()
plt.tight_layout()
plt.show()


## Moving Average

In [ ]:
from pycaret.utils import version
version()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.dates as mdates

# Load financial data
FinIns = yf.download('NG=F', start='2021-12-22', end='2025-01-30')
FinIns_close = FinIns[['Close']].dropna().sort_index()

# Ensure business day frequency
FinIns_close = FinIns_close.asfreq('B').interpolate()

# Split the data into train and test
train = FinIns_close.iloc[:-24].copy()
test = FinIns_close.iloc[-24:].copy()

# ✅ Calculate Moving Averages
FinIns['MA100'] = FinIns['Close'].rolling(window=100).mean()
FinIns['EWMA100'] = FinIns['Close'].ewm(span=100).mean()

# ✅ Figure 1: Full Dataset Forecasts
plt.figure(figsize=(14, 6))  # ✅ Create a new figure

# ✅ Plot actual data
plt.plot(train.index, train['Close'], label='Train', color='blue')
plt.plot(test.index, test['Close'], label='Test', color='grey', linewidth=2, alpha=0.7)

# ✅ Plot moving averages
plt.plot(FinIns.index, FinIns['MA100'], label='100-Day MA', color='black', linestyle='-', alpha=0.8)
plt.plot(FinIns.index, FinIns['EWMA100'], label='100-Day EWMA', color='brown', linestyle='-', alpha=0.8)

# ✅ Plot forecasts
plt.plot(y_hat_naive.index, y_hat_naive['naive'], label='Naive Forecast', color='red', linestyle='--', alpha=0.8)
plt.plot(y_hat_snaive.index, y_hat_snaive['snaive'], label='Seasonal Naive Forecast', color='green', linestyle='--', alpha=0.8)
plt.plot(y_hat_mean.index, y_hat_mean['mean'], label='Mean Forecast', color='orange', linestyle='--', alpha=0.8)
plt.plot(y_hat_drift.index, y_hat_drift['drift'], label='Drift Forecast', color='purple', linestyle='--', alpha=0.8)

# ✅ Ensure x-axis displays the full dataset range
full_start = min(train.index.min(), test.index.min(), FinIns.index.min())  
full_end = max(train.index.max(), test.index.max(), FinIns.index.max())  
plt.xlim([full_start, full_end])  

# ✅ Set up correct date formatting
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))  
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator(interval=3))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  
plt.xticks(rotation=45)  

# ✅ Add title and labels
plt.title('Forecasting Benchmarks: Full Dataset with Moving Averages', fontsize=16)
plt.ylabel('Closing Price', fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)

# ✅ Show the first plot
plt.show()

# ✅ Figure 2: Last Year Forecasts (Zoomed)
plt.figure(figsize=(14, 6))  

# ✅ Extract last year's range
last_year_start = max(test.index[-1] - pd.DateOffset(years=1), train.index[0])  
last_train = train.loc[train.index >= last_year_start] if not train.empty else pd.DataFrame()
last_test = test.loc[test.index >= last_year_start] if not test.empty else pd.DataFrame()
last_ma100 = FinIns.loc[FinIns.index >= last_year_start, 'MA100']
last_ewma100 = FinIns.loc[FinIns.index >= last_year_start, 'EWMA100']

print(f"Test Data Available: {not last_test.empty}")  

# ✅ Plot actual data (Train & Test Split)
if not last_train.empty:
    plt.plot(last_train.index, last_train['Close'], label='Train', color='blue')
if not last_test.empty:
    plt.plot(last_test.index, last_test['Close'], label='Test', color='grey', linewidth=2, alpha=0.7)  

# ✅ Plot moving averages
if not last_ma100.empty:
    plt.plot(last_ma100.index, last_ma100, label='100-Day MA', color='black', linestyle='-', alpha=0.8)
if not last_ewma100.empty:
    plt.plot(last_ewma100.index, last_ewma100, label='100-Day EWMA', color='brown', linestyle='-', alpha=0.8)

# ✅ Plot forecasts
plt.plot(y_hat_naive.loc[last_year_start:].index, y_hat_naive.loc[last_year_start:]['naive'], 
         label='Naive Forecast', color='red', linestyle='--')
plt.plot(y_hat_snaive.loc[last_year_start:].index, y_hat_snaive.loc[last_year_start:]['snaive'], 
         label='Seasonal Naive Forecast', color='green', linestyle='--')
plt.plot(y_hat_mean.loc[last_year_start:].index, y_hat_mean.loc[last_year_start:]['mean'], 
         label='Mean Forecast', color='orange', linestyle='--')
plt.plot(y_hat_drift.loc[last_year_start:].index, y_hat_drift.loc[last_year_start:]['drift'], 
         label='Drift Forecast', color='purple', linestyle='--')

# ✅ Ensure x-axis displays only last year's range
plt.xlim([last_year_start, test.index[-1]])

# ✅ Set up correct date formatting
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  
plt.xticks(rotation=45)  

# ✅ Add title and labels
plt.title('Benchmark Forecasts (Last Year) with Moving Averages', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Closing Price', fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)

# ✅ Show the second plot
plt.show()


## Data Analysis

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import matplotlib.dates as mdates
from pycaret.time_series import TSForecastingExperiment
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error

# ✅ Load Financial Data with Exception Handling
try:
    FinIns = yf.download('EC', start='2021-12-22', end='2025-01-30')
    if FinIns.empty:
        raise ValueError("No data retrieved from Yahoo Finance. Check ticker or date range.")
except Exception as e:
    print(f"Error fetching data: {e}")
    exit()

FinIns_close = FinIns[['Close']].dropna()

# ✅ Ensure Business Day Frequency & Interpolation for Missing Values
FinIns_close = FinIns_close.asfreq('B').interpolate()

# ✅ Compute Moving Averages
FinIns_close['MA100'] = FinIns_close['Close'].rolling(window=100, min_periods=1).mean()
FinIns_close['EWMA100'] = FinIns_close['Close'].ewm(span=100, adjust=False).mean()

train = FinIns_close[['Close']].iloc[:-24].copy()  # Keep only 'Close'
test = FinIns_close[['Close']].iloc[-24:].copy()

# ✅ Remove the second level of column names if they have multiple levels
if isinstance(test.columns, pd.MultiIndex):
    test.columns = test.columns.get_level_values(0)

if isinstance(train.columns, pd.MultiIndex):
    train.columns = train.columns.get_level_values(0)

# ✅ Reset MultiIndex (If applicable)
if isinstance(train.index, pd.MultiIndex):
    train = train.reset_index(level=0, drop=True)  # Drop 'Ticker' level

# ✅ Ensure DateTime Index
train.index = pd.to_datetime(train.index)

# ✅ Initialize PyCaret
exp = TSForecastingExperiment()
exp.setup(data=train, target='Close', fh=24, coverage=0.90, session_id=123)

# ✅ Statistical Analysis & Plots
exp.check_stats()
for plot_type in ['train_test_split', 'acf', 'pacf', 'decomp']:
    exp.plot_model(plot=plot_type, data_kwargs={'nlags': 36} if 'acf' in plot_type else {'type': 'multiplicative'})

[*********************100%***********************]  1 of 1 completed


,Description,Value
0,session_id,123
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(787, 1)"
5,Transformed data shape,"(787, 1)"
6,Transformed train set shape,"(763, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


## Exponential Smoothing

In [6]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import yfinance as yf
import sys
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from pycaret.time_series import TSForecastingExperiment

def load_data(ticker, start_date, end_date):
    """
    Loads and preprocesses historical financial data.

    Args:
        ticker (str): Stock symbol.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: DataFrame with interpolated 'Close' prices.
    """
    try:
        data = yf.download(ticker, start=start_date, end=end_date)

        if 'Close' not in data.columns:
            raise ValueError("No 'Close' price found in the data.")

        # Select Close price and drop NaNs
        data = data[['Close']].dropna()

        # Ensure business-day frequency
        data = data.asfreq('B')

        # Step 1: Forward Fill Missing Data
        data['Close'] = data['Close'].ffill()

        # Step 2: Apply Time-based Interpolation
        data['Close'] = data['Close'].interpolate(method='time')

        # Step 3: Apply Spline Interpolation as Fallback (Order 3 for smoothness)
        data['Close'] = data['Close'].interpolate(method='spline', order=3)

        # Handle MultiIndex Columns
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        return data

    except Exception as e:
        print(f"Error loading data: {e}")
        return None


# Function to split train-test data
def split_data(data, test_size=24):
    train, test = data.iloc[:-test_size], data.iloc[-test_size:]
    if train.index.freq is None:
        train = train.asfreq(pd.infer_freq(train.index) or 'B')
    return train, test

# Function to evaluate model
def evaluate_model(y_true, y_pred):
    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred)
    }
    print("\nModel Evaluation Metrics:")
    print("-" * 40)
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")
    return metrics

# Function to plot residuals
def plot_residuals(model, experiment):
    residuals = model.predict_residuals(y=experiment.get_config("y_train"))
    residuals.plot(title='Residuals of SES', figsize=(10, 5))
    plt.show()

# Function to plot all models on the same graph
def plot_all_forecasts(y_test, predictions, labels):
    plt.figure(figsize=(10, 5))
    
    if isinstance(y_test.index, pd.PeriodIndex):
        y_test.index = y_test.index.to_timestamp()
    
    plt.plot(y_test.index, y_test, label='Actual', marker='o')
    
    for y_pred, label in zip(predictions, labels):
        if isinstance(y_pred.index, pd.PeriodIndex):
            y_pred.index = y_pred.index.to_timestamp()
        
        plt.plot(y_pred.index, y_pred, linestyle='dashed', marker='x', label=label)
    
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Time')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid()
    plt.show()

# Load and preprocess data
FinIns_close = load_data('EC', '2021-12-22', '2025-01-30')
train, test = split_data(FinIns_close)

# Initialize PyCaret experiment
exp = TSForecastingExperiment()
exp.setup(data=train, target="Close", fh=24, coverage=0.90, session_id=42)

# Train models
ses_model = exp.create_model("exp_smooth", trend=None, seasonal=None, sp=None, cross_validation=False)
hw_linear = exp.create_model("exp_smooth", trend="add", seasonal=None, cross_validation=False)
hw_add = exp.create_model("exp_smooth", trend="add", seasonal="add", sp=12, cross_validation=False)
hw_mult = exp.create_model("exp_smooth", trend="add", seasonal="mul", sp=12, cross_validation=False)
hw_damped = exp.create_model("exp_smooth", damped_trend=True, trend="add", seasonal="mul", sp=12, cross_validation=False)

# Evaluate on test of (TRAIN) data
y_test = exp.get_config("y_test")
y_pred_ses = exp.predict_model(ses_model)
y_pred_hw = exp.predict_model(hw_linear)
y_pred_hw_add = exp.predict_model(hw_add)
y_pred_hw_mult = exp.predict_model(hw_mult)
y_pred_hw_damped = exp.predict_model(hw_damped)

if y_test is not None:
    predictions = [y_pred_ses, y_pred_hw, y_pred_hw_add, y_pred_hw_mult, y_pred_hw_damped]
    labels = ["SES", "Holt-Winter-Linear", "Holt-Winter-additive", "Holt-Winter-Multiplicative", "Holt-Winter-Damped"]
    #plot_all_forecasts(y_test, predictions, labels)
else:
    print("No test data available for evaluation.")

# Evaluate on TEST data
y_forecast_ses = exp.predict_model(ses_model, fh=48)
y_forecast_hw = exp.predict_model(hw_linear, fh=48)
y_forecast_hw_add = exp.predict_model(hw_add, fh=48)
y_forecast_hw_mult = exp.predict_model(hw_mult, fh=48)
y_forecast_hw_damped = exp.predict_model(hw_damped, fh=48)

if test is not None:
    predictions = [y_forecast_ses, y_forecast_hw, y_forecast_hw_add, y_forecast_hw_mult, y_forecast_hw_damped]
    labels = ["SES", "Holt-Winter-Linear", "Holt-Winter-additive", "Holt-Winter-Multiplicative", "Holt-Winter-Damped"]
    #plot_all_forecasts(test, predictions, labels)
else:
    print("No test data available for evaluation.")

# PyCaret forecast plots
#exp.plot_model([ses_model, hw_add, hw_mult], plot='insample', data_kwargs={'labels': ["SES", "Holt-Winter-additive", "Holt-Winter-Multiplicative"]})
exp.plot_model([ses_model, hw_add, hw_mult], plot='forecast', data_kwargs={'fh': 24, 'labels': ["SES", "Holt-Winter-additive", "Holt-Winter-Multiplicative"]})

# Compare models
exp.compare_models(include=[ses_model, hw_add, hw_mult, hw_damped], cross_validation=True)

# Plot diagnostics
exp.plot_model(estimator=hw_mult, plot="diagnostics")

print("It is done")

[*********************100%***********************]  1 of 1 completed


,Description,Value
0,session_id,42
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(787, 1)"
5,Transformed data shape,"(787, 1)"
6,Transformed train set shape,"(763, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4614,0.4184,0.4071,0.4783,0.0524,0.0506,-2.4082


,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4651,0.4220,0.4104,0.4824,0.0529,0.0510,-2.4662


,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4380,0.4022,0.3865,0.4598,0.0498,0.0481,-2.1488


...

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Exponential Smoothing,0.4380,0.4022,0.3865,0.4598,0.0498,0.0481,-2.1488


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Exponential Smoothing,0.4361,0.4005,0.3847,0.4579,0.0496,0.0479,-2.1230


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Exponential Smoothing,0.4335,0.3980,0.3824,0.4550,0.0493,0.0476,-2.0838


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
3,Exponential Smoothing,0.5051,0.4777,0.4306,0.5397,0.0501,0.0481,-0.9947,0.1267
0,Exponential Smoothing,0.5175,0.4934,0.4412,0.5573,0.0514,0.0493,-1.1318,3.4800
2,Exponential Smoothing,0.5229,0.4950,0.4457,0.5591,0.0517,0.0496,-1.1233,1.5133
1,Exponential Smoothing,0.5255,0.4980,0.4479,0.5624,0.0520,0.0499,-1.1477,2.0900


It is done
